In [2]:
from collections import namedtuple
import unyt as u
import numpy as np

In [6]:
from prepic import lwfa

In [3]:
Cetal = namedtuple(
    "Cetal",
    [
        "npe",
        "kp",
        "f_number",
        "focal_distance",
        "beam_diameter",
        "w0",
        "fwhm",
        "zR",
        "a0",
        "ɛL",
        "τL",
        "intensity",
        "power",
    ],
)

In [4]:
param = Cetal(
    npe=1.5e18 / u.cm ** 3,
    kp=0.2304711 * 1 / u.micrometer,
    f_number=24.9912 * u.dimensionless,
    focal_distance=3.2 * u.meter,
    beam_diameter=128.045071 * u.mm,
    w0=18 * u.micrometer,
    fwhm=21.193380405278543 * u.micrometer,
    zR=1.27234502 * u.mm,
    a0=4.076967454355432 * u.dimensionless,
    ɛL=7.7 * u.joule,
    τL=40 * u.femtosecond,
    intensity=3.553314404474785e19 * u.watt / u.cm ** 2,
    power=180.84167614968285 * u.terawatt,
)

In [5]:
def get_plasma(parameters):
    bubble_r = (2 * np.sqrt(parameters.a0) / parameters.kp)

    return lwfa.Plasma(
        n_pe=parameters.npe,
        laser=lwfa.Laser.from_a0(
            a0=parameters.a0, ɛL=parameters.ɛL, beam=lwfa.GaussianBeam(w0=parameters.w0)
        ),
        bubble_radius=bubble_r
    )

In [6]:
cetal_plasma = get_plasma(param)

In [32]:
cetal_plasma.laser.beam

<GaussianBeam(18.0 µm, 0.8 µm)>

In [20]:
cetal_plasma.laser.E0

unyt_quantity(16362.40354854, 'MV/mm')

In [23]:
match = lwfa.matched_laser_plasma(param.a0)

Scaling laws valid up to a0c=67.1 dimensionless


In [26]:
match.η

unyt_quantity(0.1228936, 'dimensionless')

In [28]:
sim = lwfa.Simulation(cetal_plasma)

In [31]:
sim.__dict__

{'plasma': <Plasma(1.5e+18 cm**(-3), <Laser(7.7 J, 39.99999999999999 fs, <GaussianBeam(18.0 µm, 0.8 µm)>)>, 17.521938162559955 sqrt(dimensionless)*µm)>,
 'L': unyt_quantity(109.04942675, 'µm'),
 'ppc': 8,
 'Δx': unyt_quantity(0.43389388, 'µm'),
 'Δy': unyt_quantity(0.43389388, 'µm'),
 'Δz': unyt_quantity(0.04, 'µm'),
 'nx': 251,
 'ny': 251,
 'nz': 2726,
 'npart': 1373925808,
 'dt': unyt_quantity(0.13342564, 'fs'),
 't_interact': unyt_quantity(45613.58478821, 'fs'),
 'nstep': 341865}

In [34]:
sim.L

unyt_quantity(109.04942675, 'µm')

In [35]:
sim.Δx

unyt_quantity(0.43389388, 'µm')

In [36]:
sim.Δz

unyt_quantity(0.04, 'µm')

In [37]:
sim.nx

251

In [38]:
sim.nz

2726

In [39]:
sim.npart

1373925808

In [40]:
sim.nstep

341865

1. gas-jet with $L_{\text{acc}} = 3$ mm, $n_{\text{pe}} = (1-3) \times 10^{18}$ cm${}^{-3}$
2. capillary with $L_{\text{acc}} = (3-10)$ cm, $n_{\text{pe}} = (3-7) \times 10^{17}$ cm${}^{-3}$

- $w_0 = 2 \sigma_{\text{rms}}$, and experiments say $\sigma_{\text{rms}} = 7$ $\mu$m, so $w_0 = 14$ $\mu$m
- $\varepsilon_L = 3$ J, $\tau_L = 30$ fs, $I_0 = 3 \times 10^{19}$ W/cm${}^{2}$, $a_0=3.4$

In [3]:
npe_jet = 3e18 / u.cm ** 3
l_acc_jet = 3 * u.mm
#
npe_capil = 3e17 / u.cm ** 3
l_acc_capil = 5 * u.cm

In [7]:
beam_frasc = lwfa.GaussianBeam(w0=15.56 * u.micrometer)
laser_frasc = lwfa.Laser(ɛL=3.0 * u.joule, τL=30 * u.femtosecond, beam=beam_frasc)

In [17]:
plasma_jet = lwfa.Plasma(n_pe=npe_jet, laser=laser_frasc, propagation_distance=l_acc_jet)
print(plasma_jet)

Plasma with nₚ=3.0e+18 cm**(-3) (1.72e-03 dimensionless nc), ωₚ=0.098 1/fs, kₚ=0.326 1/µm, λₚ=19.3 µm, Ewb=166.6 MV/mm
Pc=9.9 TW, Ldeph=4.38 mm, Ldepl=5.22 mm, ΔE=460.6 MeV over Lacc=3.00 mm
for laser beam with w0=15.6 µm (FWHM=18.3 µm), zᵣ=0.95 mm, λL=0.80 µm, kL=7.854 1/µm, ωL=2.355 1/fs, ɛL=3.0 J, τL=30.0 fs, P₀=93.9 TW
I₀=2.5e+19 W/cm**2, a₀=3.4 dimensionless, E₀=1.4e+04 MV/mm


In [16]:
plasma_capil = lwfa.Plasma(
    n_pe=npe_capil, laser=laser_frasc, propagation_distance=l_acc_capil
)
print(plasma_capil)

Plasma with nₚ=3.0e+17 cm**(-3) (1.72e-04 dimensionless nc), ωₚ=0.031 1/fs, kₚ=0.103 1/µm, λₚ=61.0 µm, Ewb=52.7 MV/mm
Pc=98.7 TW, Ldeph=138.49 mm, Ldepl=52.22 mm, ΔE=2427.6 MeV over Lacc=50.00 mm
for laser beam with w0=15.6 µm (FWHM=18.3 µm), zᵣ=0.95 mm, λL=0.80 µm, kL=7.854 1/µm, ωL=2.355 1/fs, ɛL=3.0 J, τL=30.0 fs, P₀=93.9 TW
I₀=2.5e+19 W/cm**2, a₀=3.4 dimensionless, E₀=1.4e+04 MV/mm


In [11]:
lwfa.Simulation(plasma_jet)

<str(<Plasma(3e+18 cm**(-3), <Laser(3.0 J, 30.0 fs, <GaussianBeam(15.56 µm, 0.8 µm)>)>, None)>, 77.10958914102495 µm, 8 dimensionless)>

In [12]:
lwfa.Simulation(plasma_capil)

<str(<Plasma(3e+17 cm**(-3), <Laser(3.0 J, 30.0 fs, <GaussianBeam(15.56 µm, 0.8 µm)>)>, None)>, 243.84193112542548 µm, 8 dimensionless)>

In [15]:
matched_frasc = lwfa.matched_laser_plasma(a0=3.4 * u.dimensionless)
print(matched_frasc)

Plasma with nₚ=3.8e+18 cm**(-3) (2.20e-03 dimensionless nc), ωₚ=0.110 1/fs, kₚ=0.368 1/µm, λₚ=17.1 µm, Ewb=188.3 MV/mm
Pc=7.7 TW, Ldeph=3.03 mm, Ldepl=3.03 mm, ΔE=526.3 MeV over Lacc=3.03 mm
for laser beam with w0=10.0 µm (FWHM=11.8 µm), zᵣ=0.39 mm, λL=0.80 µm, kL=7.854 1/µm, ωL=2.355 1/fs, ɛL=0.9 J, τL=22.3 fs, P₀=38.9 TW
I₀=2.5e+19 W/cm**2, a₀=3.4 dimensionless, E₀=1.4e+04 MV/mm
N=1.6e+09 dimensionless electrons, Q=258.0 pC, η=0.147 dimensionless
